
# Meme Virality Across Platforms: Notebook
**Group 23: Hanna , Lingchen Chen, Feihong Tian, Akshara Bruno** 

**Research question:**  
*How do the visual template (meme format) and caption framing jointly influence cross-platform virality and sentiment?*

This notebook is designed to:

1. Collect meme data from Reddit and Bluesky.  
2. Preprocess images and captions.  
3. Cluster meme templates based on image content.  
4. Analyze caption framing and sentiment.  
5. Model virality as a function of template, caption, and platform.  
6. Produce the main figures and tables.



## 0. How to run this notebook

1. Create a folder structure like:

```
project_root/
  data/
    raw/
    processed/
  images/
  models/
  notebook.ipynb
```

2. Place the cleaned datasets used in the paper into `data/processed/`  
   (e.g., `reddit_memes.csv`, `bluesky_memes.csv`).

3. (Optional) To re-scrape data from the APIs, follow the instructions in Section 2.

4. Run all cells from top to bottom.

If some external APIs or models cannot be accessed (e.g., CLIP model download),  
the corresponding cells are clearly marked so you can skip them and still see the precomputed results.


## 1. Imports and global configuration

In [ ]:
!pip install -r requirements.txt #needs to be changed. modules from requirements.txt should be added manually

/Users/aksharabruno/Documents/M Sc/VU/Social Web/Meme Analysis Project/.venv/bin/pip: line 2: /Users/aksharabruno/Documents/M Sc/VU/Social Web/Meme Project/.venv/bin/python: No such file or directory
/Users/aksharabruno/Documents/M Sc/VU/Social Web/Meme Analysis Project/.venv/bin/pip: line 2: exec: /Users/aksharabruno/Documents/M Sc/VU/Social Web/Meme Project/.venv/bin/python: cannot execute: No such file or directory


In [2]:
# Standard libraries
import os
import json
import math
import random
from pathlib import Path
from datetime import datetime

# Data & analysis
import pandas as pd
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Machine learning / embeddings / clustering
try:
    from sklearn.cluster import KMeans
    from sklearn.manifold import TSNE
except ImportError:
    print("scikit-learn is not installed. Install it to enable clustering and t-SNE visualizations.")

# Deep learning / CLIP (for image embeddings)
try:
    import torch
    from PIL import Image
    from transformers import CLIPProcessor, CLIPModel
except ImportError:
    print("torch / transformers / pillow not installed. Install them to enable image embeddings.")

# NLP
try:
    import nltk
    from nltk.sentiment import SentimentIntensityAnalyzer
except ImportError:
    print("nltk not installed. Install it to enable VADER sentiment analysis.")

# Global paths
PROJECT_ROOT = Path('.').resolve()
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
PROCESSED_DIR = DATA_DIR / "processed"
IMAGES_DIR = PROJECT_ROOT / "images"

for p in [DATA_DIR, RAW_DIR, PROCESSED_DIR, IMAGES_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("Project root:", PROJECT_ROOT)


/Users/aksharabruno/Documents/M Sc/VU/Social Web/Meme Analysis Project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Project root: /Users/aksharabruno/Documents/M Sc/VU/Social Web/Meme Analysis Project



## 2. Data collection (overview)

We collect meme posts from:

- **Reddit** – using the Reddit API (e.g. PRAW or Pushshift-like interface).  
- **Bluesky** – using the ATProto API.

For each platform we aim to obtain:

- Post ID, author, timestamp  
- Caption / title text  
- Image URL(s)  
- Engagement statistics (upvotes / likes, comments / replies, reposts)  
- Platform indicator and, for Reddit, subreddit information  

In practice, the actual scraping code may be placed in a separate script to avoid  
API credential issues inside the notebook. Here we provide a minimal example skeleton.


### 2.1 Reddit scraping skeleton (to run once, offline)

Getting data

In [8]:
import praw
import pandas as pd
from pathlib import Path
import datetime
from time import sleep

# Configuration
date_feb_2024_utc = datetime.datetime(2024, 2, 1, tzinfo=datetime.timezone.utc)
TIMESTAMP_FEB_2024 = date_feb_2024_utc.timestamp()
MIN_SCORE = 50  # Adjust based on what you consider "viral"

RAW_DIR = Path("data/raw")
reddit_path = RAW_DIR / "reddit_memes_raw.csv"

reddit = praw.Reddit(
    client_id="Dhu5BDkroWpRBQ",
    client_secret="ojQeyIZ793E7I6fRFILOiLdJzdGNcg",
    user_agent="meme-research-notebook"
)

# Target meme-heavy subreddits
target_subreddits = ["memes", "dankmemes", "AdviceAnimals", "MemeEconomy", "me_irl"]

posts = []
current_time_iso = datetime.datetime.utcnow().isoformat()
seen_ids = set()

print("🚀 Starting Reddit data collection...")
print(f"   Collecting from Feb 2024 onwards, minimum score: {MIN_SCORE}")

for subreddit_name in target_subreddits:
    print(f"\n   Processing r/{subreddit_name}...")
    subreddit = reddit.subreddit(subreddit_name)
    
    # Collect from multiple time filters to maximize coverage
    for time_filter in ['year', 'month']:
        print(f"      - Fetching top posts from past {time_filter}...")
        
        for submission in subreddit.top(time_filter=time_filter, limit=None):
            # Skip if already seen
            if submission.id in seen_ids:
                continue
                
            # Apply date filter
            if submission.created_utc < TIMESTAMP_FEB_2024:
                continue
                
            # Apply score filter
            if submission.score < MIN_SCORE:
                continue
            
            # Only collect image posts
            if not submission.url.endswith(('.jpg', '.png', '.gif', '.jpeg')):
                continue
            
            seen_ids.add(submission.id)
            
            author_name = "[deleted]"
            if submission.author is not None:
                author_name = submission.author.name
            
            posts.append({
                "platform": "reddit",
                "meme_template": "Unknown",  # Will need manual tagging later
                "post_id": submission.id,
                "subreddit": subreddit_name,
                "author_handle": author_name,
                "caption": submission.title,
                "image_url": submission.url,
                "likes": submission.score,
                "upvote_ratio": submission.upvote_ratio,
                "replies": submission.num_comments,
                "created_at": datetime.datetime.fromtimestamp(
                    submission.created_utc, tz=datetime.timezone.utc
                ).isoformat(),
                "indexed_at": current_time_iso,
                "post_url": f"https://www.reddit.com{submission.permalink}"
            })
        
        sleep(2)  # Be nice to Reddit's API
    
    print(f"      Collected {len([p for p in posts if p['subreddit'] == subreddit_name])} posts from r/{subreddit_name}")

print(f"\n✅ Total unique posts collected: {len(posts)}")

# Save to CSV
RAW_DIR.mkdir(parents=True, exist_ok=True)
reddit_df = pd.DataFrame(posts)
reddit_df.to_csv(reddit_path, index=False)
print(f"✅ Saved to {reddit_path}")

/var/folders/k5/sv2gpgh93ks3whjc1lnjkh380000gn/T/ipykernel_97541/2584464678.py:25: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_time_iso = datetime.datetime.utcnow().isoformat()


🚀 Starting Reddit data collection...

   Processing r/memes...
      - Fetching top posts from past year...
      - Fetching top posts from past month...
      Collected 453 posts from r/memes

   Processing r/dankmemes...
      - Fetching top posts from past year...
      - Fetching top posts from past month...
      Collected 467 posts from r/dankmemes

   Processing r/AdviceAnimals...
      - Fetching top posts from past year...
      - Fetching top posts from past month...
      Collected 480 posts from r/AdviceAnimals

   Processing r/MemeEconomy...
      - Fetching top posts from past year...
      - Fetching top posts from past month...
      Collected 145 posts from r/MemeEconomy

   Processing r/me_irl...
      - Fetching top posts from past year...
      - Fetching top posts from past month...
      Collected 439 posts from r/me_irl

✅ Total unique posts collected: 1984
✅ Saved to data/raw/reddit_memes_raw.csv


Download the images

In [9]:
import requests
from pathlib import Path
import pandas as pd
from time import sleep

# Load your collected data
df = pd.read_csv('data/raw/reddit_memes_raw.csv')

# Create directory for images
img_dir = Path('images/reddit_images')
img_dir.mkdir(parents=True, exist_ok=True)

print(f"Downloading {len(df)} images...")

for idx, row in df.iterrows():
    try:
        # Download image
        response = requests.get(row['image_url'], timeout=10)
        if response.status_code == 200:
            # Get file extension
            ext = row['image_url'].split('.')[-1].split('?')[0]  # Remove query params
            if ext not in ['jpg', 'jpeg', 'png', 'gif']:
                ext = 'jpg'
            
            # Save with post_id as filename
            filepath = img_dir / f"{row['post_id']}.{ext}"
            with open(filepath, 'wb') as f:
                f.write(response.content)
            
            # Update dataframe with local path
            df.at[idx, 'image_path'] = str(filepath)
            
        if (idx + 1) % 50 == 0:
            print(f"   Downloaded {idx + 1}/{len(df)} images...")
            sleep(1)  # Be nice to servers
            
    except Exception as e:
        print(f"Failed to download {row['post_id']}: {e}")

# Save updated dataframe
df.to_csv('data/raw/reddit_memes_raw.csv', index=False)
print("✅ Download complete!")

   Downloaded 50/1984 images...
   Downloaded 100/1984 images...
   Downloaded 150/1984 images...
   Downloaded 200/1984 images...
   Downloaded 250/1984 images...
   Downloaded 300/1984 images...
   Downloaded 350/1984 images...
   Downloaded 400/1984 images...
   Downloaded 450/1984 images...
   Downloaded 500/1984 images...
   Downloaded 550/1984 images...
   Downloaded 600/1984 images...
   Downloaded 650/1984 images...
   Downloaded 700/1984 images...
   Downloaded 750/1984 images...
   Downloaded 800/1984 images...
   Downloaded 850/1984 images...
   Downloaded 900/1984 images...
   Downloaded 950/1984 images...
   Downloaded 1000/1984 images...
   Downloaded 1050/1984 images...
   Downloaded 1100/1984 images...
   Downloaded 1150/1984 images...
   Downloaded 1200/1984 images...
   Downloaded 1250/1984 images...
   Downloaded 1300/1984 images...
   Downloaded 1350/1984 images...
   Downloaded 1400/1984 images...
   Downloaded 1450/1984 images...
   Downloaded 1500/1984 images...


### 2.2 Bluesky scraping skeleton (to run once, offline)

In [1]:
import sys
!{sys.executable} -m pip install atproto


[notice] A new release of pip available: 22.2.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
#Step 1: log in
print("Trying to login...")

from atproto import Client

client = Client()

# Try both options
identifiers = [
    "linnnnlnl.bsky.social",  # handle without @
    "treasureclc@gmail.com",  # your email if handle doesn't work
    "@linnnnlnl.bsky.social",  # handle with @ (less likely to work)
]

for identifier in identifiers:
    print(f"\nTrying to login with: {identifier}")
    try:
        session = client.login(identifier, "qvip-timq-o322-hnzz")
        print(f"✅ SUCCESS with {identifier}")
        print(f"   Logged in as: {session.handle}")
        break
    except Exception as e:
        print(f"❌ Failed: {e}")

# Now that we're logged in, let's understand post structure
from atproto import Client, models

# We already have a logged-in client 
# Let's test the search with the authenticated client
params = models.AppBskyFeedSearchPosts.Params(
    q="change my mind",
    limit=3  # Get 3 posts to see variety
)

feed = client.app.bsky.feed.search_posts(params)

print(f"\n📊 Found {len(feed.posts)} posts for 'change my mind'")

'''
# explore post structure
for i, post in enumerate(feed.posts):
    print(f"\n{'='*60}")
    print(f"POST {i+1}")
    print('='*60)
    
    # Basic info
    print(f"URI (unique ID): {post.uri}")
    print(f"Author: {post.author.handle}")
    
    # Text content
    if hasattr(post, 'record'):
        text = post.record.text[:150]
        print(f"Text: {text}..." if len(post.record.text) > 150 else f"Text: {post.record.text}")
        print(f"Created at: {post.record.created_at}")
    
    # Check for images
    has_images = False
    if hasattr(post, 'embed') and hasattr(post.embed, 'images'):
        image_count = len(post.embed.images)
        print(f"Has images: YES ({image_count} images)")
        for img_idx, img in enumerate(post.embed.images[:2]):  # Show first 2
            if hasattr(img, 'image'):
                print(f"  Image {img_idx+1}: {img.image}")
        has_images = True
    else:
        print("Has images: NO")
    
    # Engagement metrics
    print(f"Likes: {getattr(post, 'like_count', 0)}")
    print(f"Reposts: {getattr(post, 'repost_count', 0)}")
    print(f"Replies: {getattr(post, 'reply_count', 0)}")
    
    # Quick summary for our project
    print(f"\n✅ FOR OUR PROJECT: This post has {'images' if has_images else 'NO images'} and text for analysis")
    '''

Trying to login...

Trying to login with: linnnnlnl.bsky.social
❌ Failed: Response(success=False, status_code=401, content=XrpcError(error='AuthenticationRequired', message='Invalid identifier or password'), headers={'date': 'Mon, 08 Dec 2025 16:14:34 GMT', 'content-type': 'application/json; charset=utf-8', 'content-length': '77', 'connection': 'keep-alive', 'x-powered-by': 'Express', 'access-control-allow-origin': '*', 'ratelimit-limit': '10', 'ratelimit-remaining': '9', 'ratelimit-reset': '1765296874', 'ratelimit-policy': '10;w=86400', 'etag': 'W/"4d-98r3hvgolnybv7tgksQiZbSE7Zg"', 'vary': 'Accept-Encoding'})

Trying to login with: treasureclc@gmail.com
✅ SUCCESS with treasureclc@gmail.com
   Logged in as: linnninl.bsky.social

📊 Found 3 posts for 'change my mind'


'\n# explore post structure\nfor i, post in enumerate(feed.posts):\n    print(f"\n{\'=\'*60}")\n    print(f"POST {i+1}")\n    print(\'=\'*60)\n    \n    # Basic info\n    print(f"URI (unique ID): {post.uri}")\n    print(f"Author: {post.author.handle}")\n    \n    # Text content\n    if hasattr(post, \'record\'):\n        text = post.record.text[:150]\n        print(f"Text: {text}..." if len(post.record.text) > 150 else f"Text: {post.record.text}")\n        print(f"Created at: {post.record.created_at}")\n    \n    # Check for images\n    has_images = False\n    if hasattr(post, \'embed\') and hasattr(post.embed, \'images\'):\n        image_count = len(post.embed.images)\n        print(f"Has images: YES ({image_count} images)")\n        for img_idx, img in enumerate(post.embed.images[:2]):  # Show first 2\n            if hasattr(img, \'image\'):\n                print(f"  Image {img_idx+1}: {img.image}")\n        has_images = True\n    else:\n        print("Has images: NO")\n    \n    # 

In [6]:
# Step 2: Enhanced collection for all 10 meme templates with corrected image extraction
import time
import pandas as pd
from datetime import datetime, timezone
import sys

print("🎯 COMPREHENSIVE MEME DATA COLLECTION")
print("=====================================")
print(f"Collecting 10 meme templates at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# research constraints
SINCE_DATE = datetime(2024, 2, 1, tzinfo=timezone.utc)  # Step 1: only keep posts since Feb 1, 2024
MIN_LIKES = 100  # Step 2: like threshold 

# All 10 meme templates
ALL_MEME_TEMPLATES = [
    "change my mind",
    "drake hotline bling", 
    "distracted boyfriend",
    "two button choice",
    "expanding brain",
    "woman yelling at cat",
    "surprised pikachu",
    "mocking spongebob",
    "this is fine",
    "is this a pigeon"
]

# Alternative search queries for each template
TEMPLATE_SEARCH_VARIANTS = {
    "change my mind": ["change my mind meme", "changemymind", "#changemymind"],
    "drake hotline bling": ["drake hotline bling meme", "drake meme", "drake reaction"],
    "distracted boyfriend": ["distracted boyfriend meme", "distracted boyfriend", "looking at other girl"],
    "two button choice": ["two button meme", "two buttons", "which button"],
    "expanding brain": ["expanding brain meme", "brain expanding", "5d chess"],
    "woman yelling at cat": ["woman yelling at cat meme", "yelling cat meme", "woman cat restaurant"],
    "surprised pikachu": ["surprised pikachu meme", "pikachu surprised", "pikachu face"],
    "mocking spongebob": ["mocking spongebob meme", "spongebob mock", "mockingspongebob"],
    "this is fine": ["this is fine meme", "dog burning house", "everything is fine"],
    "is this a pigeon": ["is this a pigeon meme", "butterfly meme", "is this a butterfly"]
}

all_meme_data = []
unique_post_ids = set()  # To avoid duplicates

# Create processed directory if it doesn't exist
from pathlib import Path
PROJECT_ROOT = Path('.').resolve()
DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

def extract_image_url(embed_image):
    """Extract image URL from Bluesky ViewImage object with multiple fallbacks"""
    try:
        # Method 1: Check for fullsize attribute (newer structure)
        if hasattr(embed_image, 'fullsize'):
            return str(embed_image.fullsize)
        
        # Method 2: Check for image attribute with ref.link (older structure)
        if hasattr(embed_image, 'image'):
            if hasattr(embed_image.image, 'ref') and hasattr(embed_image.image.ref, 'link'):
                return str(embed_image.image.ref.link)
        
        # Method 3: Check for thumb attribute and convert to fullsize
        if hasattr(embed_image, 'thumb'):
            thumb_url = str(embed_image.thumb)
            # Try to convert thumbnail URL to fullsize
            if 'feed_thumbnail' in thumb_url:
                return thumb_url.replace('feed_thumbnail', 'feed_fullsize')
            # Try other common thumbnail to fullsize conversions
            if '/thumb/' in thumb_url:
                return thumb_url.replace('/thumb/', '/fullsize/')
            # Just return the thumb URL as fallback
            return thumb_url
        
        # Method 4: Check for direct link in various attributes
        for attr in ['link', 'url', 'href', 'src']:
            if hasattr(embed_image, attr):
                url = getattr(embed_image, attr)
                if url and isinstance(url, str) and url.startswith('http'):
                    return url
        
        # Method 5: Try to get from the object's dictionary representation
        if hasattr(embed_image, '__dict__'):
            for key, value in embed_image.__dict__.items():
                if isinstance(value, str) and value.startswith('http') and any(ext in value.lower() for ext in ['.jpg', '.jpeg', '.png', '.gif', '.webp']):
                    return value
        
        print(f"  ⚠️ Could not extract image URL from: {dir(embed_image)[:10]}...")
        return None
        
    except Exception as e:
        print(f"  ⚠️ Error extracting image URL: {e}")
        return None

for template in ALL_MEME_TEMPLATES:
    print(f"\n🔍 Searching for: '{template}'")
    print("-" * 40)
    
    template_posts = []
    search_queries = TEMPLATE_SEARCH_VARIANTS.get(template, [template, f"{template} meme"])
    
    for search_idx, search_query in enumerate(search_queries[:3]):  # Try first 3 variations
        print(f"  Query {search_idx+1}: '{search_query}'")
        
        try:
            # Search with pagination - get up to 100 posts per query
            cursor = None
            total_collected = 0
            
            for page in range(1, 100):  # Try up to 4 pages (25 posts per page = 100 total)
                params = models.AppBskyFeedSearchPosts.Params(
                    q=search_query,
                    limit=25,  # Max per request
                    cursor=cursor
                )
                
                feed = client.app.bsky.feed.search_posts(params)
                
                if not feed or not hasattr(feed, 'posts') or not feed.posts:
                    print(f"    Page {page}: No results")
                    break
                
                page_collected = 0
                for post in feed.posts:
                    # Skip if we've seen this post before
                    if post.uri in unique_post_ids:
                        continue
                    # 🔴 filter by created_at >= 2024-02-01
                    created_raw = getattr(post.record, 'created_at', '')
                    created_dt = None
                    if created_raw:
                        try:
                            # Bluesky timestamps look like '2024-11-29T13:45:12.345Z'
                            created_dt = datetime.fromisoformat(created_raw.replace('Z', '+00:00'))
                        except Exception:
                            created_dt = None
                
                    if created_dt is not None and created_dt < SINCE_DATE:
                        # too old → skip this post
                        continue
                
                    # 🔴 get likes once & filter
                    likes = getattr(post, 'like_count', 0)
                    if likes < MIN_LIKES:
                        # not popular enough according to our filter → skip
                        continue
                        
                    # Check if post has images
                    has_image = (hasattr(post, 'embed') and 
                                hasattr(post.embed, 'images') and 
                                post.embed.images and 
                                len(post.embed.images) > 0)
                    
                    if has_image:
                        try:
                            # Extract image URL using our helper function
                            image_url = extract_image_url(post.embed.images[0])
                            
                            if image_url:
                                # Get alt text
                                alt_text = ""
                                if hasattr(post.embed.images[0], 'alt'):
                                    alt_text = post.embed.images[0].alt
                                
                                # Extract data for analysis
                                post_data = {
                                    'platform': 'bluesky',
                                    'meme_template': template,
                                    'search_query': search_query,
                                    'post_id': post.uri,
                                    'author_handle': post.author.handle,
                                    'author_display': getattr(post.author, 'display_name', ''),
                                    'caption': getattr(post.record, 'text', ''),
                                    'alt_text': alt_text,
                                    'image_url': image_url,
                                    'likes': getattr(post, 'like_count', 0),
                                    'reposts': getattr(post, 'repost_count', 0),
                                    'replies': getattr(post, 'reply_count', 0),
                                    'created_at': getattr(post.record, 'created_at', ''),
                                    'indexed_at': getattr(post, 'indexed_at', ''),
                                    'post_url': f"https://bsky.app/profile/{post.author.handle}/post/{post.uri.split('/')[-1]}",
                                    'collection_timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                                }
                                
                                template_posts.append(post_data)
                                unique_post_ids.add(post.uri)
                                page_collected += 1
                                total_collected += 1
                        
                        except Exception as e:
                            print(f"    ⚠️ Error processing image: {e}")
                            # Debug: print the structure to understand what we're dealing with
                            if page_collected == 0:  # Only print once per page for debugging
                                print(f"    Debug - embed.images[0] type: {type(post.embed.images[0])}")
                                print(f"    Debug - embed.images[0] attributes: {[attr for attr in dir(post.embed.images[0]) if not attr.startswith('_')][:10]}")
                            continue
                
                print(f"    Page {page}: Found {len(feed.posts)} posts, {page_collected} with images")
                
                # Check for next page
                if hasattr(feed, 'cursor') and feed.cursor:
                    cursor = feed.cursor
                    time.sleep(0.5)  # Rate limiting between pages
                else:
                    break
            
            print(f"    Total from this query: {total_collected} new meme posts")
            
        except Exception as e:
            print(f"    Error with query '{search_query}': {e}")
            continue
        
        time.sleep(1)  # Rate limiting between queries
    
    # Add to overall collection
    all_meme_data.extend(template_posts)
    print(f"📊 Total for '{template}': {len(template_posts)} meme posts")
    
    # Save intermediate results after each template
    if template_posts:
        template_df = pd.DataFrame(template_posts)
        template_file = PROCESSED_DIR / f"memes_{template.replace(' ', '_')}_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
        template_df.to_csv(template_file, index=False, encoding='utf-8')
        print(f"💾 Saved template data to: {template_file}")

# Create final DataFrame
if all_meme_data:
    df_comprehensive = pd.DataFrame(all_meme_data)


    print(f"\n{'='*60}")
    print("🎉 COLLECTION COMPLETE!")
    print(f"{'='*60}")
    print(f"Total meme posts collected: {len(df_comprehensive)}")
    print(f"Unique posts: {len(unique_post_ids)}")
    
    # Save comprehensive dataset
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    output_file = PROCESSED_DIR / f"all_memes_comprehensive_{timestamp}.csv"
    df_comprehensive.to_csv(output_file, index=False, encoding='utf-8')
    
    print(f"\n💾 SAVED COMPREHENSIVE DATASET:")
    print(f"   {output_file}")
    
    # Show statistics
    print(f"\n📊 BREAKDOWN BY TEMPLATE:")
    template_counts = df_comprehensive['meme_template'].value_counts()
    for template, count in template_counts.items():
        print(f"   {template}: {count} posts")
    
    print(f"\n📊 ENGAGEMENT STATISTICS:")
    print(f"   Average likes: {df_comprehensive['likes'].mean():.1f}")
    print(f"   Average reposts: {df_comprehensive['reposts'].mean():.1f}")
    print(f"   Average replies: {df_comprehensive['replies'].mean():.1f}")
    
    print(f"\n📊 SAMPLE DATA (first 3 rows):")
    print(df_comprehensive[['meme_template', 'author_handle', 'caption', 'likes', 'reposts']].head(3).to_string(index=False))
    
    # Additional analysis
    print(f"\n📊 ADDITIONAL INSIGHTS:")
    print(f"   Posts with images: {len(df_comprehensive)}")
    print(f"   Unique authors: {df_comprehensive['author_handle'].nunique()}")
    print(f"   Date range: {df_comprehensive['created_at'].min()} to {df_comprehensive['created_at'].max()}")
    
else:
    print("\n❌ No data collected. Need to adjust search strategy.")

🎯 COMPREHENSIVE MEME DATA COLLECTION

🔍 Searching for: 'change my mind'
----------------------------------------
  Query 1: 'change my mind meme'
    Error with query 'change my mind meme': name 'models' is not defined
  Query 2: 'changemymind'
    Error with query 'changemymind': name 'models' is not defined
  Query 3: '#changemymind'
    Error with query '#changemymind': name 'models' is not defined
📊 Total for 'change my mind': 0 meme posts

🔍 Searching for: 'drake hotline bling'
----------------------------------------
  Query 1: 'drake hotline bling meme'
    Error with query 'drake hotline bling meme': name 'models' is not defined
  Query 2: 'drake meme'
    Error with query 'drake meme': name 'models' is not defined
  Query 3: 'drake reaction'
    Error with query 'drake reaction': name 'models' is not defined
📊 Total for 'drake hotline bling': 0 meme posts

🔍 Searching for: 'distracted boyfriend'
----------------------------------------
  Query 1: 'distracted boyfriend meme'
  

In [28]:
# Step 3: Download images for all collected memes
def download_all_meme_images(df):
    """Download images for comprehensive dataset"""
    import requests
    from PIL import Image
    import io
    
    print("\n📥 DOWNLOADING ALL MEME IMAGES")
    print("=" * 60)
    
    # Create images directory
    IMAGES_DIR = PROJECT_ROOT / "images"
    IMAGES_DIR.mkdir(exist_ok=True)
    
    download_count = 0
    error_count = 0
    df = df.copy()  # Work on a copy
    
    for idx, row in df.iterrows():
        try:
            # Skip if no image URL
            if not row['image_url'] or pd.isna(row['image_url']):
                error_count += 1
                continue
            
            # Create organized folder structure
            template_folder = row['meme_template'].replace(' ', '_').lower()[:20]
            template_dir = IMAGES_DIR / template_folder
            template_dir.mkdir(exist_ok=True)
            
            # Create descriptive filename
            author_safe = row['author_handle'].replace('.bsky.social', '').replace('.', '_')[:15]
            date_part = row['created_at'][:10] if row['created_at'] else 'unknown'
            filename = f"{template_folder}_{author_safe}_{date_part}_{idx}.jpg"
            filepath = template_dir / filename
            
            # Skip if already downloaded
            if filepath.exists():
                df.at[idx, 'image_path'] = str(filepath)
                download_count += 1
                continue
            
            # Download image
            print(f"  Downloading {template_folder}/{filename}...")
            
            headers = {
                'User-Agent': 'Meme Research Project v1.0 (Academic)',
                'Accept': 'image/*'
            }
            
            response = requests.get(row['image_url'], headers=headers, timeout=20)
            
            if response.status_code == 200:
                # Save image
                with open(filepath, 'wb') as f:
                    f.write(response.content)
                
                # Verify it's a valid image
                try:
                    img = Image.open(filepath)
                    img.verify()  # Verify it's a valid image
                    
                    # Store metadata
                    df.at[idx, 'image_path'] = str(filepath)
                    df.at[idx, 'image_width'], df.at[idx, 'image_height'] = img.size
                    df.at[idx, 'image_format'] = img.format
                    
                    download_count += 1
                    print(f"    ✓ {img.size} ({img.format})")
                    
                except Exception as img_error:
                    print(f"    ⚠️ Invalid image: {img_error}")
                    filepath.unlink(missing_ok=True)
                    error_count += 1
            
            else:
                print(f"    ❌ HTTP {response.status_code}")
                error_count += 1
            
            # Rate limiting
            time.sleep(0.2)
            
        except Exception as e:
            print(f"  ❌ Error row {idx}: {str(e)[:100]}")
            error_count += 1
    
    return df, download_count, error_count

# Download images if we have data
if 'df_comprehensive' in locals() and not df_comprehensive.empty:
    print(f"\n🖼️  Starting image download for {len(df_comprehensive)} posts...")
    
    df_with_images, downloaded, errors = download_all_meme_images(df_comprehensive)
    
    print(f"\n📊 IMAGE DOWNLOAD SUMMARY:")
    print(f"   Successfully downloaded: {downloaded}")
    print(f"   Errors: {errors}")
    print(f"   Success rate: {downloaded/(downloaded+errors)*100:.1f}%")
    
    # Save dataset with image paths
    final_output = PROCESSED_DIR / f"all_memes_with_images_{timestamp}.csv"
    df_with_images.to_csv(final_output, index=False, encoding='utf-8')
    
    print(f"\n💾 FINAL DATASET SAVED TO:")
    print(f"   {final_output}")
    
    # Create a summary report
    summary_file = PROCESSED_DIR / f"collection_summary_{timestamp}.txt"
    with open(summary_file, 'w') as f:
        f.write("MEME DATA COLLECTION SUMMARY\n")
        f.write("=" * 40 + "\n\n")
        f.write(f"Collection date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Total posts collected: {len(df_with_images)}\n")
        f.write(f"Images successfully downloaded: {downloaded}\n")
        f.write(f"Download errors: {errors}\n\n")
        
        f.write("BY TEMPLATE:\n")
        for template, count in df_with_images['meme_template'].value_counts().items():
            f.write(f"  {template}: {count}\n")
    
    print(f"\n📄 Summary report saved to: {summary_file}")

Processing 46 meme posts...

📥 DOWNLOADING MEME IMAGES
    ✓ Success: (500, 485)
    ✓ Success: (577, 432)
    ✓ Success: (973, 720)
    ✓ Success: (577, 432)
    ✓ Success: (1206, 920)
    ✓ Success: (577, 432)
    ✓ Success: (640, 480)
    ✓ Success: (914, 672)
    ✓ Success: (2000, 2000)
    ✓ Success: (1061, 1200)
    ✓ Success: (577, 432)
    ✓ Success: (577, 432)
    ✓ Success: (640, 480)
    ✓ Success: (1054, 2000)
    ✓ Success: (1111, 833)
    ✓ Success: (640, 480)
    ✓ Success: (577, 432)
    ✓ Success: (1999, 1500)
    ✓ Success: (586, 678)
    ✓ Success: (1080, 384)
    ✓ Success: (500, 701)
    ✓ Success: (500, 1020)
    ✓ Success: (1028, 1066)
    ✓ Success: (500, 701)
    ✓ Success: (750, 759)
    ✓ Success: (1414, 2000)
    ✓ Success: (500, 525)
    ✓ Success: (1373, 1938)
    ✓ Success: (645, 960)
    ✓ Success: (1179, 1179)
    ✓ Success: (513, 499)
    ✓ Success: (508, 2000)
    ✓ Success: (1102, 1561)
    ✓ Success: (660, 936)
    ✓ Success: (500, 701)
    ✓ Succes

## 3. Load processed data

In [ ]:

# In the final project, you will provide cleaned CSV files in data/processed.
# Here we load them; if they do not exist yet, we show an empty DataFrame
# so that the notebook can still run without errors.

reddit_path = PROCESSED_DIR / "reddit_memes.csv"
bluesky_path = PROCESSED_DIR / "bluesky_memes.csv"

if reddit_path.exists():
    reddit_df = pd.read_csv(reddit_path)
else:
    reddit_df = pd.DataFrame()
    print("WARNING: reddit_memes.csv not found in data/processed. Using empty DataFrame.")

if bluesky_path.exists():
    bluesky_df = pd.read_csv(bluesky_path)
else:
    bluesky_df = pd.DataFrame()
    print("WARNING: bluesky_memes.csv not found in data/processed. Using empty DataFrame.")

print("Reddit shape:", reddit_df.shape)
print("Bluesky shape:", bluesky_df.shape)



## 4. Data cleaning and harmonization

Goals:

- Merge Reddit and Bluesky into a unified schema.  
- Standardize timestamp format.  
- Create unified fields for:
  - `caption` (title/text)  
  - `image_url`  
  - `likes`, `comments`, `shares` (or platform equivalents)  
  - `platform`  

We also filter to posts that actually contain images and belong to the meme templates of interest.


In [ ]:

# Example harmonization skeleton

def unify_reddit(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out["platform"] = "reddit"
    out["caption"] = out["title"].fillna("") + " " + out["selftext"].fillna("")
    out["image_url"] = out["url"]
    out["likes"] = out["score"]
    out["comments"] = out["num_comments"]
    out["shares"] = np.nan  # Reddit has no explicit "shares"
    out["created_at"] = pd.to_datetime(out["created_utc"], unit="s")
    return out[["platform", "caption", "image_url", "likes", "comments", "shares", "created_at"]]

def unify_bluesky(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    out = df.copy()
    out["platform"] = "bluesky"
    out["caption"] = out["text"]
    out["image_url"] = out.get("image_urls", "")
    out["likes"] = out.get("like_count", np.nan)
    out["comments"] = out.get("reply_count", np.nan)
    out["shares"] = out.get("repost_count", np.nan)
    out["created_at"] = pd.to_datetime(out["created_at"])
    return out[["platform", "caption", "image_url", "likes", "comments", "shares", "created_at"]]

unified_frames = []
for df, fn in [(reddit_df, unify_reddit), (bluesky_df, unify_bluesky)]:
    unified_frames.append(fn(df))

full_df = pd.concat(unified_frames, ignore_index=True) if unified_frames else pd.DataFrame()
print("Unified dataset shape:", full_df.shape)
full_df.head()



## 5. Image download and caching

We download each meme image to `images/` and keep a mapping from post ID to local file path.

This enables:

- reproducible CLIP embeddings  
- offline analysis  
- visual inspection per template cluster


In [ ]:

import hashlib
import requests

def download_image(url: str, dest_dir: Path) -> Path | None:
    if not isinstance(url, str) or not url.startswith("http"):
        return None
    fname = hashlib.md5(url.encode("utf-8")).hexdigest() + ".jpg"
    fpath = dest_dir / fname
    if fpath.exists():
        return fpath
    try:
        resp = requests.get(url, timeout=10)
        resp.raise_for_status()
        with open(fpath, "wb") as f:
            f.write(resp.content)
        return fpath
    except Exception as e:
        # In the final project you may log this instead of printing
        print("Failed to download", url, "->", e)
        return None

if not full_df.empty and "image_url" in full_df.columns:
    local_paths = []
    for url in full_df["image_url"]:
        local_paths.append(download_image(url, IMAGES_DIR))
    full_df["image_path"] = local_paths
else:
    print("No image URLs available yet; skipping download step.")



## 6. Image embeddings and template clustering

We use a pre-trained CLIP model to obtain an embedding vector for each meme image,  
then cluster memes into template-like groups using k-means.

Later, we will label clusters with interpretable template names (e.g., "change my mind").


In [ ]:

CLIP_ENABLED = False  # set to True when dependencies and GPU/CPU resources are available

if CLIP_ENABLED and not full_df.empty and "image_path" in full_df.columns:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    embed_list = []
    valid_idx = []
    for idx, path in enumerate(full_df["image_path"]):
        if path and isinstance(path, str) and os.path.exists(path):
            image = Image.open(path).convert("RGB")
            inputs = processor(images=image, return_tensors="pt").to(device)
            with torch.no_grad():
                emb = model.get_image_features(**inputs)
            embed_list.append(emb.cpu().numpy().flatten())
            valid_idx.append(idx)

    if embed_list:
        embeds = np.vstack(embed_list)
        n_clusters = 8  # adjust after exploration
        kmeans = KMeans(n_clusters=n_clusters, random_state=0)
        cluster_labels = kmeans.fit_predict(embeds)

        full_df.loc[valid_idx, "template_cluster"] = cluster_labels
    else:
        print("No embeddings computed (no valid images).")
else:
    print("CLIP embedding step disabled or no images available.")


### 6.1 Cluster visualization (t-SNE / example images)

In [ ]:

# TODO: After computing template_cluster, create:
# - t-SNE scatter plot colored by cluster
# - montage of example images per cluster

# Example placeholder for t-SNE:
if not full_df.empty and "template_cluster" in full_df.columns:
    print("Template clusters available. Add t-SNE visualization here.")
else:
    print("Template clusters not yet computed.")



## 7. Caption processing, sentiment and framing

Here we:

- Clean captions (lowercase, remove URLs, etc.)  
- Compute sentiment scores (e.g., using VADER).  
- Derive simple framing features:
  - presence of question marks / exclamations  
  - first-person vs. third-person pronouns  
  - political vs. non-political keywords  
- (Optional) Use topic modeling or more advanced framing classifiers.


In [ ]:

import re

def basic_clean(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

if not full_df.empty:
    full_df["caption_clean"] = full_df["caption"].apply(basic_clean)
else:
    print("Unified dataset empty; skipping caption cleaning.")


In [ ]:

# Sentiment analysis with VADER (if available)
try:
    nltk.data.find("sentiment/vader_lexicon.zip")
except LookupError:
    try:
        nltk.download("vader_lexicon")
    except Exception as e:
        print("Could not download VADER lexicon:", e)

if not full_df.empty:
    try:
        sia = SentimentIntensityAnalyzer()
        sentiment_scores = full_df["caption_clean"].apply(lambda t: sia.polarity_scores(t)["compound"])
        full_df["sentiment"] = sentiment_scores
    except Exception as e:
        print("Sentiment analysis failed:", e)
else:
    print("Unified dataset empty; skipping sentiment computation.")


In [ ]:

# Simple framing features (placeholder)
if not full_df.empty:
    full_df["has_question"] = full_df["caption_clean"].str.contains("\?")
    full_df["has_exclaim"] = full_df["caption_clean"].str.contains("!")
    full_df["length_tokens"] = full_df["caption_clean"].str.split().apply(len)

    # Example: detect potentially political captions using keyword list
    political_keywords = ["election", "president", "government", "policy", "vote"]
    pattern = "|".join(political_keywords)
    full_df["is_political"] = full_df["caption_clean"].str.contains(pattern)
else:
    print("Unified dataset empty; skipping framing features.")



## 8. Virality metrics

We operationalize **virality** as a function of:

- Likes / upvotes  
- Comments / replies  
- Shares / reposts  
- (Optional) Time-normalized variants (e.g., score per hour)

We then examine how virality varies across:

- meme templates (clusters)  
- caption sentiment and framing features  
- platforms (Reddit vs Bluesky)


In [ ]:

if not full_df.empty:
    # Example composite virality score (you may refine this):
    def compute_virality(row):
        likes = row.get("likes", 0) or 0
        comments = row.get("comments", 0) or 0
        shares = row.get("shares", 0) or 0
        return likes + 2*comments + 3*shares

    full_df["virality_raw"] = full_df.apply(compute_virality, axis=1)

    # Log-transform to reduce skew
    full_df["virality_log1p"] = np.log1p(full_df["virality_raw"])
else:
    print("Unified dataset empty; skipping virality computation.")



## 9. Modeling: how template and caption framing influence virality

We fit simple models to understand joint effects of:

- Template cluster  
- Sentiment  
- Framing features  
- Platform  

on virality.

Possible approaches:

- Linear regression / regularized regression  
- Mixed-effects models (platform as random effect)  
- Tree-based models (e.g., RandomForestRegressor)  

Below we show a basic regression skeleton.


In [ ]:

try:
    from sklearn.linear_model import LinearRegression
except ImportError:
    print("scikit-learn not installed; install it to run the regression.")
else:
    if not full_df.empty and "virality_log1p" in full_df.columns:
        model_df = full_df.dropna(subset=["virality_log1p"]).copy()

        # Create simple numeric features
        model_df["platform_is_reddit"] = (model_df["platform"] == "reddit").astype(int)
        model_df["template_cluster"] = model_df.get("template_cluster", -1).fillna(-1).astype(int)

        feature_cols = [
            "platform_is_reddit",
            "template_cluster",
            "sentiment",
            "has_question",
            "has_exclaim",
            "is_political",
            "length_tokens",
        ]

        # Convert booleans to ints
        for col in ["has_question", "has_exclaim", "is_political"]:
            if col in model_df.columns:
                model_df[col] = model_df[col].astype(int)

        X = model_df[feature_cols].fillna(0)
        y = model_df["virality_log1p"]

        if len(model_df) > 10:
            reg = LinearRegression()
            reg.fit(X, y)
            print("Fitted LinearRegression with coefficients:")
            for col, coef in zip(feature_cols, reg.coef_):
                print(f"  {col}: {coef:.3f}")
        else:
            print("Not enough data points to fit a regression model.")
    else:
        print("No virality data available for modeling.")



## 10. Main figures and tables

This section should reproduce all main figures, charts and tables used in the paper, such as:

1. Distribution of virality by platform  
2. Sentiment vs. virality  
3. Template cluster vs. virality  
4. Interaction plots (e.g., sentiment–virality by platform)  
5. Example memes per template cluster (qualitative case studies)

Below we include placeholders for you to fill with the actual visualizations.


In [ ]:

# 10.1 Virality distributions by platform
if not full_df.empty and "virality_log1p" in full_df.columns:
    full_df.boxplot(column="virality_log1p", by="platform")
    plt.title("Virality (log1p) by platform")
    plt.suptitle("")
    plt.xlabel("Platform")
    plt.ylabel("log1p(virality)")
    plt.show()
else:
    print("No virality data available for plotting.")


In [ ]:

# 10.2 Sentiment vs. virality scatter (placeholder)
if not full_df.empty and "sentiment" in full_df.columns and "virality_log1p" in full_df.columns:
    plt.scatter(full_df["sentiment"], full_df["virality_log1p"], alpha=0.3)
    plt.xlabel("Sentiment (compound)")
    plt.ylabel("log1p(virality)")
    plt.title("Sentiment vs. virality")
    plt.show()
else:
    print("Sentiment or virality not available for plotting.")



## 11. Qualitative case studies

Here you can:

- Show a small set of example memes from key template clusters  
- Compare how the **same template** is used on Reddit vs. Bluesky  
- Discuss differences in caption framing and user reaction  

You can use `IPython.display.display(Image(...))` to show example images.


In [ ]:

# TODO: Implement helper to display example memes per cluster / platform.
# from IPython.display import display
# display(Image(filename=some_path))



## 12. Limitations and ethical considerations

In this markdown cell, discuss:

- Sampling biases (which subreddits / hashtags you used)  
- API limitations (rate limits, deleted posts, etc.)  
- Representativeness of Reddit and Bluesky users  
- Potential ethical issues (e.g., sensitive content, privacy)  
- Constraints of using automated sentiment / framing models on memes



## 13. Summary and link to paper

Summarize the main findings that this notebook reproduces, in a few bullet points:

- How visual templates relate to virality  
- How caption framing and sentiment relate to virality  
- Key differences between Reddit and Bluesky  

Reference the corresponding sections / figures in your paper so graders can easily cross-check.
